In [1]:
from pyomo.opt import SolverFactory
from pyomo.core import Var
from pyomo.core import Param
from operator import itemgetter
import pandas as pd
from datetime import datetime
import os

In [2]:
###Segment C.1
yr = 2016
run_no = 1
os.getcwd()

'F:\\NCUC_Camb\\Camb_model\\to_git\\Model'

In [3]:
###Segment C.2
from PowNetModel import model

Gen_Nodes: 12


In [4]:
###Segment C.3
instance = model.create_instance('pownet_data_camb_2016.dat')

In [5]:
###Segment C.4
opt = SolverFactory("gurobi") ##SolverFactory("cplex")
opt.options["threads"] = 1
H = instance.HorizonHours
K=range(1,H+1)
start = 1 ##1 to 364
end  = 366 ##2 to 366

In [6]:
###Segment C.5
#Space to store results
on=[]
switch=[]

mwh=[]
hydro=[]
solar=[]
wind=[]

hydro_import=[]

srsv=[]
nrsv=[]

vlt_angle=[]

for day in range(start,end):
    for z in instance.d_nodes:
     #load Demand and Reserve time series data
        for i in K:
            instance.HorizonDemand[z,i] = instance.SimDemand[z,(day-1)*24+i]
            instance.HorizonReserves[i] = instance.SimReserves[(day-1)*24+i] 
            
    for z in instance.h_nodes:
     #load Hydropower time series data
        for i in K:
            instance.HorizonHydro[z,i] = instance.SimHydro[z,(day-1)*24+i]
            
#     for z in instance.s_nodes:
#      #load Solar time series data
#         for i in K:
#             instance.HorizonSolar[z,i] = instance.SimSolar[z,(day-1)*24+i]
            
#     for z in instance.w_nodes:
#      #load Wind time series data
#         for i in K:
#             instance.HorizonWind[z,i] = instance.SimWind[z,(day-1)*24+i]
            
    for z in instance.h_imports:
     #load Hydropower time series data
        for i in K:
            instance.HorizonHydroImport[z,i] = instance.SimHydroImport[z,(day-1)*24+i]
        
    result = opt.solve(instance) ##,tee=True to check number of variables
    instance.solutions.load_from(result)   
 
#  #The following section is for storing and sorting results
    for v in instance.component_objects(Var, active=True):
        varobject = getattr(instance, str(v))
        a=str(v)
        if a=='hydro':
      
             for index in varobject:
                 if int(index[1]>0 and index[1]<25):
                    if index[0] in instance.h_nodes:
                        hydro.append((index[0],index[1]+((day-1)*24),varobject[index].value))   

#         if a=='solar':
      
#              for index in varobject:
#                  if int(index[1]>0 and index[1]<25):
#                     if index[0] in instance.s_nodes:
#                         solar.append((index[0],index[1]+((day-1)*24),varobject[index].value))   

#         if a=='wind':
      
#              for index in varobject:
#                  if int(index[1]>0 and index[1]<25):
#                     if index[0] in instance.w_nodes:
#                         wind.append((index[0],index[1]+((day-1)*24),varobject[index].value))   
                            

        if a=='hydro_import':
      
             for index in varobject:
                 if int(index[1]>0 and index[1]<25):
                    if index[0] in instance.h_imports:
                        hydro_import.append((index[0],index[1]+((day-1)*24),varobject[index].value))   

        if a=='vlt_angle':
      
             for index in varobject:
                 if int(index[1]>0 and index[1]<25):
                    if index[0] in instance.nodes:
                        vlt_angle.append((index[0],index[1]+((day-1)*24),varobject[index].value))   

        if a=='mwh':
            for index in varobject:
                if int(index[1]>0 and index[1]<25):
                    mwh.append((index[0],index[1]+((day-1)*24),varobject[index].value))                                              

                        
        if a=='on':        
            for index in varobject:
                if int(index[1]>0 and index[1]<25):
                    on.append((index[0],index[1]+((day-1)*24),varobject[index].value))

        if a=='switch':  
            for index in varobject:
                if int(index[1]>0 and index[1]<25):
                    switch.append((index[0],index[1]+((day-1)*24),varobject[index].value))


        if a=='srsv':    
            for index in varobject:
                if int(index[1]>0 and index[1]<25):
                    srsv.append((index[0],index[1]+((day-1)*24),varobject[index].value))

                        
        if a=='nrsv':    
            for index in varobject:
                if int(index[1]>0 and index[1]<25):
                    nrsv.append((index[0],index[1]+((day-1)*24),varobject[index].value))

        
    hydro_pd=pd.DataFrame(hydro,columns=('Node','Time','Value'))
    hydro_import_pd=pd.DataFrame(hydro_import,columns=('Node','Time','Value'))
#     solar_pd=pd.DataFrame(solar,columns=('Node','Time','Value'))
#     wind_pd=pd.DataFrame(wind,columns=('Node','Time','Value'))
    
    vlt_angle_pd=pd.DataFrame(vlt_angle,columns=('Node','Time','Value'))
    
    mwh_pd=pd.DataFrame(mwh,columns=('Generator','Time','Value'))    
    on_pd=pd.DataFrame(on,columns=('Generator','Time','Value'))
    switch_pd=pd.DataFrame(switch,columns=('Generator','Time','Value'))
    srsv_pd=pd.DataFrame(srsv,columns=('Generator','Time','Value'))
    nrsv_pd=pd.DataFrame(nrsv,columns=('Generator','Time','Value'))



###to save outputs
    mwh_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_mwh.csv')
    
    hydro_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_hydro.csv')
    hydro_import_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_hydro_import.csv')
#     solar_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_solar.csv')
#     wind_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_wind.csv')
    
    vlt_angle_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_vlt_angle.csv')
    
    on_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_on.csv')
    switch_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_switch.csv')
    srsv_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_srsv.csv')
    nrsv_pd.to_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_nrsv.csv')
 
    print(day)
    print(str(datetime.now()))



1
2019-09-25 11:27:30.261767
2
2019-09-25 11:27:31.864233
3
2019-09-25 11:27:33.440461
4
2019-09-25 11:27:36.024605
5
2019-09-25 11:27:38.625716
6
2019-09-25 11:27:39.836774
7
2019-09-25 11:27:41.067847
8
2019-09-25 11:27:43.172346
9
2019-09-25 11:27:45.094815
10
2019-09-25 11:27:46.528592
11
2019-09-25 11:27:48.279698
12
2019-09-25 11:27:49.322798
13
2019-09-25 11:27:50.600152
14
2019-09-25 11:27:52.285159
15
2019-09-25 11:27:54.953879
16
2019-09-25 11:27:56.001329
17
2019-09-25 11:27:57.620418
18
2019-09-25 11:27:58.784727
19
2019-09-25 11:27:59.652963
20
2019-09-25 11:28:00.555457
21
2019-09-25 11:28:01.914653
22
2019-09-25 11:28:02.893706
23
2019-09-25 11:28:04.106434
24
2019-09-25 11:28:05.349038
25
2019-09-25 11:28:06.346963
26
2019-09-25 11:28:07.888435
27
2019-09-25 11:28:08.857137
28
2019-09-25 11:28:09.869025
29
2019-09-25 11:28:10.855026
30
2019-09-25 11:28:12.286259
31
2019-09-25 11:28:13.997070
32
2019-09-25 11:28:15.276794
33
2019-09-25 11:28:16.787619
34
2019-09-25 11:28

269
2019-09-25 11:40:05.765779
270
2019-09-25 11:40:10.059161
271
2019-09-25 11:40:14.436634
272
2019-09-25 11:40:18.784016
273
2019-09-25 11:40:22.825377
274
2019-09-25 11:40:26.472258
275
2019-09-25 11:40:30.425788
276
2019-09-25 11:40:34.822320
277
2019-09-25 11:40:39.328083
278
2019-09-25 11:40:42.959282
279
2019-09-25 11:40:46.982088
280
2019-09-25 11:40:51.702601
281
2019-09-25 11:40:55.248839
282
2019-09-25 11:40:59.310585
283
2019-09-25 11:41:03.920693
284
2019-09-25 11:41:07.959513
285
2019-09-25 11:41:11.675577
286
2019-09-25 11:41:16.276689
287
2019-09-25 11:41:20.599368
288
2019-09-25 11:41:24.987218
289
2019-09-25 11:41:31.402234
290
2019-09-25 11:41:35.618346
291
2019-09-25 11:41:40.775985
292
2019-09-25 11:41:45.881685
293
2019-09-25 11:41:49.592409
294
2019-09-25 11:41:54.074912
295
2019-09-25 11:41:58.702108
296
2019-09-25 11:42:03.038787
297
2019-09-25 11:42:06.646464
298
2019-09-25 11:42:10.285059
299
2019-09-25 11:42:14.249324
300
2019-09-25 11:42:18.354829
301
2019